# Step [1]: Prepare libraries and data
### [1.1] Include important libraries

In [5]:
import nltk
import re
import pickle
import pandas as pd
from string import punctuation
from nltk.corpus import stopwords
punctuation = punctuation + '\n'
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import RidgeClassifier
import nltk 
import string
from pyarabic.araby import strip_harakat


###  Reading data

In [6]:
ar_data = pd.read_csv(r"datasets/arabic_dataset.csv")
ar_data = ar_data.replace("diverse", "diverse news")
ar_data = ar_data.replace("culture", "diverse news")
ar_data = ar_data.replace("politic", "politics")
ar_data = ar_data.replace("technology", "tech")
ar_data = ar_data.replace("economy", "economy & business")
ar_data = ar_data.replace("internationalNews", "politics")
ar_data = ar_data[~ar_data['type'].str.contains('localnews')]
ar_data = ar_data[~ar_data['type'].str.contains('society')]


In [7]:
ar_data.sample(10)

,Unnamed: 0,text,type
4171,1247,\nيواجه النادي الصفاقسي عشية اليوم بداية من ا...,sport
2307,125,\nوصف رئيس حزب التحالف الديمقراطي، محمد الحامد...,politics
1595,668,\nقال مسؤولٌ كبير في إدارة الرئيس الأميركي دون...,politics
3250,326,\nيعقد رئيس الترجي الجرجيسي ندوة صحفية في بحر ...,sport
1085,158,\nدعا الناطق باسم القيادة العامة للقوات المسلح...,politics
3704,780,\nدارت اليوم منافسات الجولة الرابعة لمرحلة الت...,sport
2169,1242,\nتتواصل الاحتجاجات في الحسيمة المغربية لليوم ...,politics
2825,643,"\n أهابت تونس، مساء الجمعة، بكل الليبيين، ""الت...",politics
2320,138,\nفي حوار حصري أدلى به للجوهرة أف أم، وفي أول ...,politics
2798,616,\nأعلن رئيس مجلس النواب، محمد الناصر أمس الجمع...,politics


# Cleaning

In [8]:
# def delete_links(input_text):
#     pettern  = r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
#     out_text = re.sub(pettern, ' ', input_text)
#     return out_text

In [9]:
# def delete_repeated_characters(input_text):
#     pattern  = r'(.)\1{2,}'
#     out_text = re.sub(pattern, r"\1\1", input_text)
#     return out_text

In [10]:
# def replace_letters(input_text):
#     replace = {"أ": "ا","ة": "ه","إ": "ا","آ": "ا","": ""}
#     replace = dict((re.escape(k), v) for k, v in replace.items()) 
#     pattern = re.compile("|".join(replace.keys()))
#     out_text = pattern.sub(lambda m: replace[re.escape(m.group(0))], input_text)
#     return out_text

In [11]:
# def clean_text(input_text):
#     replace = r'[/(){}\[\]|@âÂ,;\?\'\"\*…؟–’،!&\+-:؛-]'
#     out_text = re.sub(replace, " ", input_text)
#     words = nltk.word_tokenize(out_text)
#     words = [word for word in words if word.isalpha()]
#     out_text = ' '.join(words)
#     return out_text

In [12]:
# def remove_vowelization(input_text):
#     vowelization = re.compile(""" ّ|َ|ً|ُ|ٌ|ِ|ٍ|ْ|ـ""", re.VERBOSE)
#     out_text = re.sub(vowelization, '', input_text)
#     return out_text

In [13]:
# def delete_stopwords(input_text):
#     stop_words = set(nltk.corpus.stopwords.words("arabic") + nltk.corpus.stopwords.words("english"))
#     tokenizer = nltk.tokenize.WhitespaceTokenizer()
#     tokens = tokenizer.tokenize(input_text)
#     wnl = nltk.WordNetLemmatizer()
#     lemmatizedTokens =[wnl.lemmatize(t) for t in tokens]
#     out_text = [w for w in lemmatizedTokens if not w in stop_words]
#     out_text = ' '.join(out_text)
#     return out_text
# # print(nltk.corpus.stopwords.words("arabic"))

In [14]:
# def text_prepare(input_text):
#     out_text = delete_links(input_text)
#     out_text = delete_repeated_characters(out_text)
#     out_text = clean_text(out_text)
#     out_text = delete_stopwords(out_text)
   
#     out_text = replace_letters(out_text)
#     out_text = remove_vowelization(out_text)
    
#     return out_text

In [15]:

#Cleaning data
arabic_punctuations = '''«»`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def clean(text):
    output = re.sub(r'\s*[A-Za-z]+\b', ' ' , text) #Remove english letters
    output = strip_harakat(output) #Remove harakat   
    translator = str.maketrans(' ',' ', punctuations_list) #remove arabic and english punctuations
    output = output.translate(translator)
    output = " ".join(output.split()) #remove extra spaces
    output = re.sub('\w*\d\w*', ' ', output)# Remove numbers
    return output


In [16]:
#Cleaning String
def apply_clean_string(input_text):
    out_text = clean(input_text)
    return out_text

In [18]:
ar_data.sample(10)

,Unnamed: 0,text,type
1930,1003,\nقالت الشرطة الأسترالية يوم الأحد إن طفلا رضي...,politics
1617,690,"\nعثر عاملون في مطار ""أورلي"" بباريس على جثة رج...",politics
3225,301,\nاكد نائب رئيس ترجي جرجيس الهاشمي عبيشو في تص...,sport
2992,68,\nينتظر فريق النجم الرياضي الساحلي قدوم الحافل...,sport
1256,329,\nأصدر قاض اتحادي في هاواي أمرا بوقف تنفيذ الح...,politics
736,137,\nتفاقم العجز التجاري لتونس خلال الأشهر الثلاث...,economy & business
426,302,\nانشغل لص بعد أن تسلّل إلى منزل واقع في ولاية...,diverse news
1790,863,\nحذر وزير الأوقاف والشؤون الدينية الفلسطيني ا...,politics
1651,724,\nقرر القضاء الفرنسي توجيه اتهامات تتعلق بجرائ...,politics
3364,440,\nإجتمع اليوم مكتب الرّابطة الوطنيّة لكرة الق...,sport


# Spliting

In [19]:
ar_data['Processed Text'] = ar_data['text'].apply(apply_clean_string)
# After label encoding we sholud change some labels to another becouse the arabic dataset labels is not the same with english dataset
ar_label_encoder = LabelEncoder()
ar_data['Category Encoded'] = ar_label_encoder.fit_transform(ar_data['type'])

In [20]:
categories = ['Economy & Business', 'Diverse News', 'Politic', 'Sport', 'Technology']

In [21]:
ar_X_train, ar_X_test, ar_y_train, ar_y_test = train_test_split(ar_data['Processed Text'], ar_data['Category Encoded'], test_size=0.2, random_state=0)

# TF-IDF 

In [22]:
arb_stopwords = set(nltk.corpus.stopwords.words("arabic"))


In [23]:
def tfidf_features(X_train, X_test, ngram_range):
    tfidf_vectorizer = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1, ngram_range))
    X_train = tfidf_vectorizer.fit_transform(X_train)
    X_test = tfidf_vectorizer.transform(X_test)
    return X_train, X_test

In [24]:
ar_features_train, ar_features_test = tfidf_features(ar_X_train, ar_X_test,2)

# TRain

In [25]:
def fit_model(model_name, ar_text=False):
    if model_name == 'ridge_model':
        model_name = RidgeClassifier()
    elif model_name == 'random_forest_model':
        model_name = RandomForestClassifier()
    elif model_name == 'logistic_regression_model':
        model_name = LogisticRegression()
    elif model_name == 'kneighbors_model':
        model_name = KNeighborsClassifier()
    elif model_name == 'decision_tree_model':
        model_name = DecisionTreeClassifier()
    elif model_name == 'gaussian_nb_model':
        model_name = GaussianNB()
    if ar_text:
        model_name.fit(ar_features_train.toarray(), ar_y_train)
        model_predictions = model_name.predict(ar_features_test.toarray())
        print("Accuracy on test: ", accuracy_score(ar_y_test, model_predictions))
   
    return model_name

In [26]:
ar_ridge_model = fit_model('ridge_model', True)
pickle.dump(ar_ridge_model, open('models/ar_ridge_model.pkl','wb'))

Accuracy on test:  0.9182530795072789


In [27]:
ar_random_forest_model = fit_model('random_forest_model', True)
pickle.dump(ar_random_forest_model, open('models/ar_random_forest_model.pkl','wb'))

Accuracy on test:  0.8432250839865622


In [28]:
ar_logistic_regression_model = fit_model('logistic_regression_model', True)
pickle.dump(ar_logistic_regression_model, open('models/ar_logistic_regression_model.pkl','wb'))

Accuracy on test:  0.8723404255319149


In [29]:
ar_kneighbors_model = fit_model('kneighbors_model', True)
pickle.dump(ar_kneighbors_model, open('models/ar_kneighbors_model.pkl','wb'))

Accuracy on test:  0.22284434490481522


In [30]:
ar_decision_tree_model = fit_model('decision_tree_model', True)
pickle.dump(ar_decision_tree_model, open('models/ar_decision_tree_model.pkl','wb'))

Accuracy on test:  0.7726763717805151


In [31]:
ar_gaussian_nb_model = fit_model('gaussian_nb_model', True)
pickle.dump(ar_gaussian_nb_model, open('models/ar_gaussian_nb_model.pkl','wb'))

Accuracy on test:  0.8622620380739082
